In [ ]:
import time
import duckdb
import numpy as np
import load_to_db
import embeddings
import db_queries
import main



# this is the file to which the sql database will be saved
db_file_c = duckdb.connect("/cluster/work/coss/anmusso/victoria/loaded_data/loaded_comments.db")
db_file_s = duckdb.connect("/cluster/work/coss/anmusso/victoria/loaded_data/loaded_submissions.db")



(We could consider changing extract_comments to also add to existing tables)

Load comments into duckdb and get embeddings 

In [ ]:

#main.load('S', '05', '2020')
#main.load('S', '06', '2020')
#main.load('C', '05', '2020')
#main.load('C', '06', '2020')

In [ ]:
#run to check number of submissions/comments loaded
start = time.time()
print("C: \n", db_file_c.sql("SELECT Count(*) FROM comments;"))
print("S: \n", db_file_s.sql('SELECT Count(*) FROM submissions;'))
print("time to compute: ", time.time()-start)


Calculate embeddings: (uncomment line)
available models: 'all-mpnet-base-v2' , 'all-MiniLM-L6-v2', 'multi-qa-mpnet-base-dot-v1'
will automatically use gpu if available
use main.py with batch system to choose GPU

In [ ]:
t= 'S' #type month and year
model_name = 'all-MiniLM-L6-v2'


start = time.time()
#num = main.embedd( t, model_name=model_name) #returns number of embeddings calculated

with open('time_tracking_2.txt', 'a') as f: #loggs time into file for comparison
    f.write(f"set: {t}, size:{num} model:{model_name}, gpu:{torch.cuda.get_device_name(0)} , time:{time.time()-start} \n")


Choose table and subreddits we want to isolate

In [ ]:
table = "submissions"
subr = ['Republican', 'democrats','healthcare', 'Feminism', 'nra', 'education', 'climatechange',
            'politics','progressive', 'The_Donald','TrueChristian','Trucks','teenagers','AskMenOver30',
            'backpacking','news','BlackLivesMatter','racism','news','usa','DefundPoliceNYC']




We can retrieve the numbers, their embeddings and corresponding attributes 
umap the embeddings and display them with their corresponding attributes

In [ ]:
import embedding_graphs.UMAP_embeddings as umap

subr_nums = db_queries.get_subreddit_numbers(table_name = table, sql_db = db_file_s, subreddits = subr)

M_embedd = main.fetch_embeddings(num_list = subr_nums, table_name=table, model_name= 'all-MiniLM-L6-v2')

atr = ['title', 'subreddit']
atr_df = db_queries.get_attributes(attributes=atr, numbers=subr_nums, table_name=table, sql_db=db_file_s)
print(atr_df)

umap.show_umap(M_embedd,atr_df,'umap try 1')

alternitavely we can retrieve the coordinates of UMAP Embedding, here we use GPU UMAP of all submissions:
(this requires more than 20GB in memory else retrieving embeddings will crash)

In [ ]:
import embedding_graphs.UMAP_embeddings as um

table = 'submissions'

subr_nums = db_queries.get_all_numbers(table_name=table, sql_db=db_file_s)
print('got ids', len(subr_nums))
subr_nums = subr_nums[0:1000000]

M_embedd = main.fetch_embeddings(num_list = subr_nums, table_name=table, model_name= 'all-MiniLM-L6-v2')
print('got embeddings', M_embedd.shape)

coordinates = um.UMAP_to_df_gpu(M_embedd, n_neighbors = 50,  n_components=3, min_dist=0.1)
del M_embedd
coordinates.columns = ["x", "y",'z']


Next we create the clusters for the new coordinates

In [1]:
import numpy as np
try_matrix = np.random.uniform(size = (1_000_000,3))

In [2]:
import cuml
scanner = cuml.cluster.hdbscan.HDBSCAN(
    min_cluster_size = 100,
    algorithm = 'ball_tree'
)
clusters = scanner.fit_predict(try_matrix)


[I] [17:39:51.990171] Unused keyword parameter: algorithm during cuML estimator initialization


we save this to a new table, which we can then join on the num with the original

In [ ]:

coordinates["num"] = subr_nums
coordinates["cluster"] = clusters

db_file_s.sql(f"DROP TABLE IF EXISTS umap_coordinates")
db_file_s.sql("CREATE TABLE umap_coordinates AS SELECT * FROM coordinates")

db_file_s.sql(f"DROP TABLE IF EXISTS submission_info")
db_file_s.sql("CREATE TABLE submission_info AS SELECT * FROM submissions JOIN coordinates ON (coordinates.num = submissions.num); ")

we can visualize the clusters to doublecheck

In [ ]:
um.show_umap_3d(coordinates=coordinates, color=clusters, title='umap coordinates', hover_data=subr_nums)

next we calculate tf-idf to find top words for each cluster.
first extract document of each cluster

In [ ]:
import topic_finding as tf 
tfidf = tf.get_tfidf()
top_words = tf.get_top_terms()
sizes = tf.get_topic_sizes()